In [1]:
from __future__ import print_function, division
import os
import sys 

spark_home = os.environ['SPARK_HOME']
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))

from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [2]:
spark = SparkSession.builder.getOrCreate()


In [3]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [4]:
lines.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)


In [6]:
wordCounts = words.groupBy("word").count()

In [7]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

In [8]:
query.awaitTermination()

KeyboardInterrupt: 